# Vision 기능
이미지를 생성하거나 편집하려면 Images API 또는 Responses API를 사용할 수 있습니다.

| **API**                  | **지원되는 사용 사례**                                  |
| ------------------------ | ----------------------------------------------- |
| **Responses API**        | 이미지를 분석하고 입력으로 사용하거나, 이미지를 출력으로 생성할 수 있습니다.     |
| **Images API**           | 이미지를 출력으로 생성하며, 선택적으로 다른 이미지를 입력으로 사용할 수도 있습니다. |

In [1]:
from dotenv import load_dotenv
load_dotenv() 

True

In [2]:
from openai import OpenAI
client = OpenAI()

## 이미지 분석 (Analyze images)

비전(Vision) 은 모델이 이미지를 “보고” 이해하는 능력을 의미합니다.  
이미지 안에 텍스트가 포함되어 있다면, 모델은 그 텍스트도 인식할 수 있습니다.  
또한 모델은 객체, 형태, 색상, 질감(texture) 등 대부분의 시각적 요소를 이해할 수 있지만, 일부 한계는 존재합니다.

### 이미지의 내용 설명
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" width=300px /> 

**이미지 입력의 세부 수준 지정 (Specify image input detail level)**

`detail` 매개변수는 모델이 이미지를 **얼마나 세밀하게 분석하고 이해할지**를 결정합니다.
이 매개변수에는 다음 세 가지 옵션이 있습니다:

* `"low"` — 낮은 세부 수준으로 처리 (빠르지만 세밀한 인식은 제한적)
* `"high"` — 높은 세부 수준으로 처리 (더 정확하지만 처리 비용이 높음)
* `"auto"` — 모델이 상황에 따라 적절한 수준을 자동으로 선택

`detail` 매개변수를 생략하면,
모델은 기본적으로 `"auto"` 모드를 사용합니다.


In [3]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

Model = "gpt-5-nano"

response = client.responses.create(
  model=Model,
  input=[
    {
      "role": "user",
      "content": [
        {"type": "input_text", "text": "이 이미지의 내용을 설명해줘. 한국어로 설명해줘."},
        {"type": "input_image", 
         "image_url": url,
         "detail": "high"   # low, high, auto
        },
      ],
    }
  ]
)

print(response.output_text)

초록빛 들판이 넓게 펼쳐진 풍경이에요. 가운데로 길게 뻗은 목재 산책로가 있고, 양옆의 tall grasses가 어우러져 자연스러운 길을 따라 걸어가고 있는 느낌입니다. 멀리 트리와 덤불들이 점점 모여 있고, 하늘은 맑은 파란색에 흩뿌린 흰 구름이 부드러운 분위기를 만들어줍니다. 전체적으로 밝고 따뜻한 오후의 평온한 자연 풍경으로 보입니다. 사람들이나 동물은 보이지 않는 조용한 공간이에요.


### Base 64로 인코딩된 이미지 업로드
로컬에 이미지 또는 이미지 세트가 있는 경우 이를 Base 64 인코딩 형식으로 모델에 전달할 수 있습니다. 다음은 이에 대한 실제 예입니다.

In [5]:
import base64

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "data/냉장고내용물.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.responses.create(
  model=Model,
  input=[
    {
      "role": "user",
      "content": [
        {"type": "input_text", "text": "이 이미지의 내용을 한국어로 설명해줘."},
        {"type": "input_image",
          "image_url": f"data:image/jpeg;base64,{base64_image}"}
      ],
    }
  ],
)

print(response.output_text)

이 이미지의 내용은 열린 냉장고 안에 신선한 식재료가 가득 담긴 모습입니다.

- 상단 선반: 과일이 빠르게 보관되어 있어요. 오렌지, 레몬, 자몽 같은 과일들과 사과가 섞여 있으며 바구니에 담긴 과일도 있습니다.
- 중간 선반: 피망, 가지, 오이, 양상추, 시금치 등 다양한 채소가 정리되어 있고, 토마토나 아보카도 같은 식재료도 함께 보입니다.
- 하단 선반/서랍: 당근, 다양한 뿌리채소와 채소류가 들어 있는 것 같고, 채소가 담긴 용기들도 보입니다.
- 도어 선반: 주스 병과 우유류 병들이 배열되어 있습니다.
- 전반적으로 아주 다채로운 색상의 신선한 과일과 채소가 가득 차 있는 모습입니다.


### 다중 이미지 입력
Chat Completions API는 base64로 인코딩된 형식이나 이미지 URL로 여러 이미지 입력을 가져와 처리할 수 있습니다. 모델은 각 이미지를 처리하고 모든 이미지의 정보를 사용하여 질문에 답합니다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" width=300px />

<img src="https://img.freepik.com/premium-photo/a-wooden-walkway-leads-to-a-green-field-with-a-mountain-in-the-background_800563-10576.jpg" width=300px />

In [6]:
response = client.responses.create(
  model=Model,
  input=[
    {
      "role": "user",
      "content": [
        {
          "type": "input_text", "text": "이 이미지에는 무엇이 들어있나요? 그들 사이에 어떤 차이가 있나요?",
        },
        {
          "type": "input_image",
          "image_url": 
            "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
        },
        {
          "type": "input_image",
          "image_url":
            "https://img.freepik.com/premium-photo/a-wooden-walkway-leads-to-a-green-field-with-a-mountain-in-the-background_800563-10576.jpg",
        },
      ],
    }
  ]
)

In [7]:
print(response.output_text)

간단히 말하면 두 이미지 모두 보드워크(나무 길)가 초록 들판을 가로지르며 멀리까지 이어지는 풍경입니다. 하늘이 넓고 탁 트인 점도 비슷합니다.

주요 차이점
- 시간대/조명: 첫 이미지는 한낮의 맑은 하늘과 선명한 파란빛, 차가운 느낌. 두 번째 이미지는 해질 무렵의 따뜻한 주황빛으로 분위기가 더 몽환적이고 강한 대비를 보임.
- 색감: 첫 이미지는 푸른 하늘과 밝은 녹색 풀이 주를 이룸. 두 번째 이미지는 황혼의 주황/노랑 빛이 들판과 보드워크에 반사되어 따뜻한 톤.
- 구성/전경 느낌: 첫 이미지는 중앙으로 길이 똑바로 뻗어나가며 넓은 들판과 얕은 구름이 특징. 두 번째 이미지는 길이가 더 근접해서 시점이 낮아 보이고, 주변 들판이 약간 더 어두워 보이는 경향이 있음. 멀리 보이는 배경은 첫 이미지에 비해 언덕이나 산처럼 보이는 지형이 덜 드러나고 더 먼 분위기.
- 분위기: 첫 이미지가 밝고 산뜻한 느낌, 두 번째 이미지가 서정적이고 여운이 남는 느낌.

원하시면 각 이미지의 특정 부분(예: 보드워크의 질감, 들판의 식생 차이)에 집중해서 더 자세히 설명해 드릴게요.


### 한계점 (Limitations)

비전(Vision) 기능을 가진 모델은 매우 강력하고 다양한 상황에서 활용할 수 있지만,
이러한 모델에도 **명확한 한계**가 있습니다.
다음은 알려진 주요 제한 사항들입니다:

---

* **의료 영상 (Medical images)**

  * CT 스캔 등 **전문 의료 영상 해석**에는 적합하지 않으며,
    의료 조언이나 진단 목적으로 사용해서는 안 됩니다.

* **비영어권 문자 (Non-English text)**

  * 일본어나 한국어처럼 **비라틴 문자**가 포함된 이미지에서는
    인식 및 이해 성능이 최적이 아닐 수 있습니다.

* **작은 글씨 (Small text)**

  * 이미지 안의 글씨는 확대하면 인식이 개선되지만,
    중요한 세부 정보가 잘리지 않도록 주의해야 합니다.

* **회전된 이미지 (Rotation)**

  * 회전되거나 거꾸로 된 텍스트나 이미지를
    잘못 해석할 가능성이 있습니다.

* **시각적 요소(Visual elements)**

  * 색상이나 선의 스타일(예: 실선, 점선, 파선 등)이
    달라지는 **그래프나 도표의 구분**을 정확히 하지 못할 수 있습니다.

* **공간 추론 (Spatial reasoning)**

  * 바둑판이나 체스처럼 **정확한 공간적 위치 파악이 필요한 작업**에는 약합니다.

* **정확도 (Accuracy)**

  * 특정 상황에서는 잘못된 설명이나 캡션을 생성할 수 있습니다.

* **이미지 형태 (Image shape)**

  * 파노라마(panoramic)나 어안(fisheye) 이미지처럼
    비정상적인 비율의 이미지를 처리하는 데 어려움이 있습니다.

* **메타데이터 및 크기 변경 (Metadata and resizing)**

  * 원본 파일 이름이나 메타데이터를 처리하지 않으며,
    분석 전에 이미지를 리사이즈하기 때문에 **원본 해상도 정보가 손실**될 수 있습니다.

* **개체 수 세기 (Counting)**

  * 이미지 내의 객체 수를 세는 작업은 **대략적인 추정치**만 제공합니다.

* **CAPTCHA**

  * **보안상의 이유**로 CAPTCHA 이미지는 시스템에서 차단되어
    분석 요청을 보낼 수 없습니다.
